In [1]:
import numpy as np
from sklearn.datasets import fetch_mldata
mnist = fetch_mldata('MNIST original')

In [2]:
mnist

{'COL_NAMES': ['label', 'data'],
 'DESCR': 'mldata.org dataset: mnist-original',
 'data': array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], dtype=uint8),
 'target': array([0., 0., 0., ..., 9., 9., 9.])}

细心发现, 从上边可以看出,data 中存放的是int 类型的数据, 但在机器学习中,一般训练数据集均为float类型,    
因此,需要将data 中的数据变为float类型

In [3]:
X = mnist.data
X.shape

(70000, 784)

In [9]:
y = mnist.target
y.shape

(70000,)

注意：mnist 数据集已经为我们分好了训练和测试数据集,     
一般默认 前60000个样本为训练数据集,后10000个样本为测试数据集

In [14]:
X_train = X[:60000,:]
y_train = y[:60000]
X_test = X[60000:,:]
y_test = y[60000:]
#将数据类型转换为float, 在本例子中,数据类型是int, 还是float 都相同,但是为保险起见,一般都将数据类型转换为float
# X_train = np.array(X[:60000,:],dtype = float)
# y_train = np.array(y[:60000], dtype = float)
# X_test = np.array(X[60000:,:], dtype = float)
# y_test = np.array(y[60000:], dtype = float)


In [15]:
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(60000, 784) (60000,) (10000, 784) (10000,)


# 直接使用KNN算法,识别手写数字

In [16]:
from sklearn.neighbors import KNeighborsClassifier
Knn = KNeighborsClassifier()
%time Knn.fit(X_train,y_train)
Knn.score(X_test, y_test)

CPU times: user 51.5 s, sys: 232 ms, total: 51.7 s
Wall time: 51.9 s


0.9688

# 先使用PCA进行降维, 然后再使用KNN,进行手写数字的识别

In [17]:
from sklearn.decomposition import PCA
pca = PCA(n_components=0.9)
pca.fit(X_train)
X_train_reduction = pca.transform(X_train)
X_test_reduction = pca.transform(X_test)

In [18]:
Knn1 = KNeighborsClassifier()
%time Knn1.fit(X_train_reduction,y_train)
Knn1.score(X_test_reduction, y_test)

CPU times: user 664 ms, sys: 0 ns, total: 664 ms
Wall time: 681 ms


0.9728

In [25]:
pca.n_components_

87

取原始数据90%的信息,可以将原始样本从784维降到87维,    
进行KNN分类时,运行速度从原来的51.5 s 降到了 664 ms毫秒,速度得到了将近10倍的提升.   
识别的准确率从 0.9688 提升到了 0.9728, 丢失了一部分信息,但是识别的准确率却更高了, 这说明,数据中可能存在噪音,我们可能是将那些不重要的信息(噪音)丢掉了.   
从而也说明了PCA 具有去噪的作用

注意1： 对于图像, 不用使用standard Scalar, 因为只有在数据尺度不同时才使用standard Scalar,而像素点的灰度的尺度相同   
注意2： sklearn 中的KNN算法 的实现方法与我们本地封装的KNN的实现方法有略微不同    
        sklearn中KNN对于样本量比较大的数据,会自动的使用树结构来加快KNN过程.   
   